In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
#loading data from csv
orders=pd.read_csv(r"..\Data\olist_orders_dataset.csv")
order_items = pd.read_csv(r"../data/olist_order_items_dataset.csv")  
reviews = pd.read_csv(r"../data/olist_order_reviews_dataset.csv") 
category_translation = pd.read_csv(r"../Data/product_category_name_translation.csv")
products=pd.read_csv(r"../Data/olist_products_dataset.csv")


In [20]:
  
print("Orders shape:", orders.shape)  
print("Order items shape:", order_items.shape)  
print("reviews shape:", reviews.shape)  
print("category translation shape:", category_translation.shape)  
print("products shape:", products.shape)  
 

Orders shape: (99441, 8)
Order items shape: (112650, 7)
reviews shape: (99224, 7)
category translation shape: (71, 2)
products shape: (32951, 9)


In [21]:
# Step 1: Merge orders + order_items
sales_data=pd.merge(orders,order_items,on="order_id",how="inner")



In [22]:
# 2. Add product details
sales_data = pd.merge(sales_data,products,on='product_id',how='left')


In [23]:
# 3. Translate categories to English
sales_data = pd.merge(sales_data,category_translation,on='product_category_name',how='left')


In [24]:
#4. Convert review dates to datetime
#reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])

#4. Sort and keep last review per order
reviews = reviews.sort_values('review_creation_date').drop_duplicates('order_id', keep='last')

# Then merge
full_data = pd.merge(
    sales_data,
    reviews[['order_id', 'review_score', 'review_comment_message']],
    on='order_id',
    how='left'
)

#print(full_data.shape)

In [25]:
#print(full_data.isnull().sum().sort_values(ascending=False))

In [26]:
#Handling Missing Values
# A. Review scores (fill unreviewed orders with median)
review_median=full_data['review_score'].median()
full_data['review_score']=full_data['review_score'].fillna(review_median)

# B. Product Weight

overall_median = full_data['product_weight_g'].median()
full_data['product_weight_g'] = full_data['product_weight_g'].fillna(overall_median)

# C. Freight values (fill with 0 if missing)
full_data['freight_value'] = full_data['freight_value'].fillna(0)


In [27]:
#2. Fix Data Types
# Convert dates
date_cols = ['order_purchase_timestamp', 'shipping_limit_date','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']
for col in date_cols:
    full_data[col] = pd.to_datetime(full_data[col], errors='coerce')

# Convert categoricals
full_data['product_category_name_english'] = full_data['product_category_name_english'].astype('category')
#print(full_data.shape)

In [28]:
#3. Handle Outliers
# A. Price outliers (cap at 99th percentile)
price_cap = full_data['price'].quantile(0.99)
full_data['price'] = full_data['price'].clip(upper=price_cap)

# B. Delivery time outliers
full_data['delivery_days'] = (full_data['order_delivered_customer_date'] - 
                             full_data['order_purchase_timestamp']).dt.days
full_data = full_data[(full_data['delivery_days'] > 0) & 
                     (full_data['delivery_days'] < 100)]  # Remove unrealistic values


In [29]:
#Create New Features
# A. Shipping efficiency
full_data['weight_to_freight_ratio'] = full_data['product_weight_g'] / (full_data['freight_value'] + 0.01)  # Avoid division by zero

# B. Review sentiment flag
full_data['positive_review'] = (full_data['review_score'] >= 4).astype(int)

# C. Purchase time features
full_data['purchase_hour'] = full_data['order_purchase_timestamp'].dt.hour
full_data['purchase_day_of_week'] = full_data['order_purchase_timestamp'].dt.day_name()


In [30]:
#5. Final Validation Checks

#  A. Check financial consistency
assert (full_data['price'] > 0).all(), "Negative prices found"
assert (full_data['freight_value'] >= 0).all(), "Negative freight values"

In [ ]:
# Get all duplicate order-product pairs
duplicates = full_data[full_data.duplicated(['order_id', 'product_id'], keep=False)]

# Show most frequent duplicates
print(f"Found {len(duplicates)} duplicate order-product pairs")
print(duplicates['order_id'].value_counts().head())
# Check if duplicates have different prices
price_variation = duplicates.groupby(['order_id', 'product_id'])['price'].std().fillna(0)
if price_variation.max() > 0:
    print("\nWarning: Some duplicates have different prices!")
    print("Aggregating prices...")
    deduplicated = full_data.groupby(['order_id', 'product_id'], as_index=False).agg({
        'price': 'sum',
        **{col: 'first' for col in full_data.columns if col not in ['order_id', 'product_id', 'price']}
    })
else:
    print("\nAll duplicates have identical prices - keeping first occurrence")
    deduplicated = full_data.drop_duplicates(['order_id', 'product_id'])

# Final validation
assert not deduplicated.duplicated(['order_id', 'product_id']).any()
#print(f"\nFinal shape: {deduplicated.shape}")


Found 16942 duplicate order-product pairs
order_id
ab14fdcfbe524636d65ee38360e22ce8    20
1b15974a0141d54e36626dca3fdc731a    20
8272b63d03f5f79c56e9e4120aec44ef    20
428a2f660dc84138d969ccd69a0ab6d5    15
9ef13efd6949e4573a18964dd1bbe7f5    15
Name: count, dtype: int64

All duplicates have identical prices - keeping first occurrence

Final shape: (100115, 30)


In [32]:
# 1. First convert the categorical columns to regular strings
full_data['product_category_name'] = full_data['product_category_name'].astype(str)
full_data['product_category_name_english'] = full_data['product_category_name_english'].astype(str)

# 2. Now replace empty strings and 'nan' with 'unknown'
full_data['product_category_name'] = full_data['product_category_name'].replace(['nan', ''], 'unknown')
full_data['product_category_name_english'] = full_data['product_category_name_english'].replace(['nan', ''], 'unknown')

# 3. Verify
'''print("Missing values after cleaning:")
print(full_data[['product_category_name', 'product_category_name_english']].isnull().sum())
print("\nCategory counts:")
print(full_data['product_category_name_english'].value_counts(dropna=False))'''


'print("Missing values after cleaning:")\nprint(full_data[[\'product_category_name\', \'product_category_name_english\']].isnull().sum())\nprint("\nCategory counts:")\nprint(full_data[\'product_category_name_english\'].value_counts(dropna=False))'

In [33]:
#save cleaned data to new csv

full_data.to_csv("../Data/cleaned_ecommerce_data.csv", index=False)
